In [2]:
# import useful libraries

import numpy as np
import itertools
from collections import Counter
from utils import load_puzzle_input, test_code
import re
from tqdm import tqdm

In [3]:
# load puzzle input
    
puzzle_input = load_puzzle_input('input.txt')

print(puzzle_input[:3])

['Sensor at x=2924811, y=3544081: closest beacon is at x=3281893, y=3687621', 'Sensor at x=2719183, y=2520103: closest beacon is at x=2872326, y=2415450', 'Sensor at x=3754787, y=3322726: closest beacon is at x=3281893, y=3687621']


### Part 1

In [4]:
# code for preparing and testing examples for part 1

example_dict_part_1 = {
    tuple(load_puzzle_input('example_1.txt')): 26
}

In [5]:
def parse_line(line):
    
    matches = re.search(r"Sensor at x=(.*), y=(.*): closest beacon is at x=(.*), y=(.*)", line)
    
    return [int(matches.group(x)) for x in range(1, 4+1)]

In [6]:
def calculate_manhattan_distance(x1, y1, x2, y2):
    
    return sum([abs(x1 - x2), abs(y1 - y2)])

In [7]:
def find_all_points_at_distance(distance, origin_x, origin_y, important_y):
    """Too slow for part 1..."""
    
    point_set = set()
    
    for new_x in range(origin_x - distance, origin_x + distance + 1):
        
        for new_y in range(origin_y - (distance - abs(origin_x - new_x)), origin_y + (distance - abs(origin_x - new_x)) + 1):
            
            if new_y == important_y:
            
                point_set.add((new_x, new_y))
            
    return point_set

In [20]:
def find_x_points_at_distance_on_y(distance, origin_x, origin_y, important_y):
    
    x_set = set()

    if important_y in range(origin_y - distance, origin_y + distance + 1):
        
        for new_x in range(origin_x - (distance - abs(origin_y - important_y)), origin_x + (distance - abs(origin_y - important_y)) + 1):
            
            x_set.add(new_x)
            
    return x_set

In [29]:
def identify_beacon_absence_points(sensor_x, sensor_y, beacon_x, beacon_y, important_y):
    """adjusted to now only store x values"""
    
    distance = calculate_manhattan_distance(sensor_x, sensor_y, beacon_x, beacon_y)
    
    # null_points = find_all_points_at_distance(distance, sensor_x, sensor_y, important_y)
    null_x_points = find_x_points_at_distance_on_y(distance, sensor_x, sensor_y, important_y)
    
    return {x for x in null_x_points if (x, important_y) != (beacon_x, beacon_y)}

In [35]:
def part_1_solution(puzzle_input, y = 2000000):
    """ note: y is 10 for test, 2000000 for live """
    
    null_point_set = set()
    
    for line in tqdm(puzzle_input):
        
        sensor_x, sensor_y, beacon_x, beacon_y = parse_line(line)
        
        for new_null_point in identify_beacon_absence_points(sensor_x, sensor_y, beacon_x, beacon_y, y):
            
            # print(f"sensor @: {(sensor_x, sensor_y)}, beacon @: {beacon_x, beacon_y}, null_point: {new_null_point}")
            
            null_point_set.add(new_null_point)
            
    return len(null_point_set)

In [33]:
# test part 1 solution

test_code(part_1_solution, example_dict_part_1)

100%|██████████| 14/14 [00:00<00:00, 81217.50it/s]

Test 0 passed: Input <('Sensor at x=2, y=18: closest beacon is at x=-2, y=15', 'Sensor at x=9, y=16: closest beacon is at x=10, y=16', 'Sensor at x=13, y=2: closest beacon is at x=15, y=3', 'Sensor at x=12, y=14: closest beacon is at x=10, y=16', 'Sensor at x=10, y=20: closest beacon is at x=10, y=16', 'Sensor at x=14, y=17: closest beacon is at x=10, y=16', 'Sensor at x=8, y=7: closest beacon is at x=2, y=10', 'Sensor at x=2, y=0: closest beacon is at x=2, y=10', 'Sensor at x=0, y=11: closest beacon is at x=2, y=10', 'Sensor at x=20, y=14: closest beacon is at x=25, y=17', 'Sensor at x=17, y=20: closest beacon is at x=21, y=22', 'Sensor at x=16, y=7: closest beacon is at x=15, y=3', 'Sensor at x=14, y=3: closest beacon is at x=15, y=3', 'Sensor at x=20, y=1: closest beacon is at x=15, y=3')> gives output <26>.

Congratulations! Looks like you cracked it! Good job!


In [36]:
part_1_solution(puzzle_input)

100%|██████████| 24/24 [00:03<00:00,  7.38it/s]


5040643

### Part 2

In [37]:
# code for preparing and testing examples for part 2

example_dict_part_2 = {
    tuple(load_puzzle_input('example_1.txt')): 56000011
}

In [38]:
def calculate_tuning_frequency(x, y):
    return (x * 4000000) + y

In [68]:
def find_x_points_at_distance_on_y_maxed(distance, origin_x, origin_y, important_y, max_x):
    
    x_set = set()

    if important_y in range(origin_y - distance, origin_y + distance + 1):
        
        for new_x in range(max(0, origin_x - (distance - abs(origin_y - important_y))), min(max_x, origin_x + (distance - abs(origin_y - important_y)) + 1)):
            
            x_set.add(new_x)
            
    return x_set

In [65]:
def identify_beacon_absence_points_maxed(sensor_x, sensor_y, beacon_x, beacon_y, important_y, max_x):
    """adjusted to now only store x values"""
    
    distance = calculate_manhattan_distance(sensor_x, sensor_y, beacon_x, beacon_y)
    
    # null_points = find_all_points_at_distance(distance, sensor_x, sensor_y, important_y)
    null_x_points = find_x_points_at_distance_on_y_maxed(distance, sensor_x, sensor_y, important_y, max_x)
    
    return {x for x in null_x_points if (x, important_y) != (beacon_x, beacon_y)}

In [87]:
def identify_all_beacon_absences(parsed_line_list, y, max):
    
    absences_set = set()
    
    for sensor_x, sensor_y, beacon_x, beacon_y in parsed_line_list:
        
        for absence_x in identify_beacon_absence_points_maxed(sensor_x, sensor_y, beacon_x, beacon_y, y, max):
            
            absences_set.add(absence_x)
            
        if beacon_y == y:
            
            absences_set.add(beacon_x)
            
    return absences_set

In [97]:
def identify_1_range_out_points(sensor_x, sensor_y, beacon_x, beacon_y, max):
    
    point_set = set()
    
    distance = calculate_manhattan_distance(sensor_x, sensor_y, beacon_x, beacon_y)

    for new_y in range(sensor_y - distance - 1, sensor_y + distance + 2):
           
        # for new_x in range(origin_x - (distance - abs(origin_y - important_y)), origin_x + (distance - abs(origin_y - important_y)) + 1):
            
        #     x_set.add(new_x)
        
        point_set.add(
            (
                sensor_x - (distance - abs(sensor_y - new_y)), 
                new_y
            )
        )
        
        point_set.add(
            (
                sensor_x + (distance - abs(sensor_y - new_y)) + 1, 
                new_y
            )
        )
            
    return point_set

In [99]:
identify_1_range_out_points(0, 0, 2, 0, 10)

{(-2, 0),
 (-1, -1),
 (-1, 1),
 (0, -3),
 (0, -2),
 (0, 2),
 (0, 3),
 (1, -3),
 (1, -2),
 (1, 2),
 (1, 3),
 (2, -1),
 (2, 1),
 (3, 0)}

In [95]:
def part_2_solution(puzzle_input, max = 4000000):
    """ note: max is 20 for test, 4000000 for live"""
    
    parsed_line_list = [parse_line(line) for line in puzzle_input]

    for y in tqdm(range(max + 1)):
        
        # absences = [list(identify_beacon_absence_points_maxed(sensor_x, sensor_y, beacon_x, beacon_y, y, max)) for sensor_x, sensor_y, beacon_x, beacon_y in parsed_line_list]
        # beacons = [beacon_x for _, _, beacon_x, beacon_y in parsed_line_list if beacon_y == y]
        # absences.append(beacons)
        # absences = [item for sublist in absences for item in sublist] # flatten list
        # absences_set = set(absences)
        
        absences_set = identify_all_beacon_absences(parsed_line_list, y, max)
        
        # print(absences_set)
        
        if len(absences_set) != max:
            
            print('here we go...')

            for x in range(max + 1):

                if x not in absences_set:
                    
                    print(f"solution: {x}, {y}")

                    return calculate_tuning_frequency(x, y)

    return None

In [93]:
# test part 2 solution

test_code(part_2_solution, example_dict_part_2)

 52%|█████▏    | 11/21 [00:00<00:00, 12151.00it/s]

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}
{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19}
here we go...
solution: 14, 11
Test 0 passed: Input <('Sensor at x=2, y=18: closest beacon is at x=-2, y=15', 'Sensor at x=9, y=16: closest beacon is at

In [96]:
part_2_solution(puzzle_input)

  0%|          | 22/4000001 [00:49<2505:07:50,  2.25s/it]


KeyboardInterrupt: 